In [ ]:
# Install Google GenAI SDK, Pillow, Gradio, and requests with compatible versions
!pip install -q requests==2.32.4 google-genai==1.40.0 pillow gradio


In [ ]:
from getpass import getpass
import os

# Securely input your Gemini API key (will not show in notebook)
api_key = getpass("Paste your Gemini API key (won't show): ")

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key

# Import the Gemini client
from google import genai
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

print("Gemini client created successfully!")


In [ ]:
import re

try:
    prompt = """You are a caption generator for student event posters and memes.
Generate 4 short, punchy social-media captions (one per line) for:
college fest, music, food, friends.
Keep each caption 1-2 short lines."""

    resp = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt
    )

    text = resp.text or ""
    print("Raw response:\n", text)

    # Parse numbered/bulleted captions
    captions = [re.sub(r'^[\-\d\.\)\s]+', '', ln).strip() for ln in text.splitlines() if ln.strip()]
    if len(captions) < 1:
        # fallback if returned as paragraph
        captions = [p.strip() for p in re.split(r'[。\n.!?]+', text) if p.strip()][:4]

    print("\nParsed captions:", captions)

except Exception as e:
    print("Error calling Gemini API:", type(e).__name__, str(e))


In [ ]:
# Download fonts for memes and posters
!wget -q -O /content/Bangers-Regular.ttf https://github.com/google/fonts/raw/main/ofl/bangers/Bangers-Regular.ttf
!wget -q -O /content/Montserrat-SemiBold.ttf https://github.com/google/fonts/raw/main/ofl/montserrat/Montserrat-SemiBold.ttf

from PIL import Image, ImageDraw, ImageFont
import textwrap, requests, io

FONT_BANGERS = "/content/Bangers-Regular.ttf"
FONT_MONTSERRAT = "/content/Montserrat-SemiBold.ttf"

print("Fonts downloaded successfully!")


In [ ]:
def draw_text_on_image(img, text, font_path=FONT_BANGERS, position='bottom', padding=40):
    img = img.convert("RGBA")
    W, H = img.size
    draw = ImageDraw.Draw(img)

    font_size = max(18, int(W / 15))
    font = ImageFont.truetype(font_path, font_size)

    avg_char_w = sum(font.getsize(c)[0] for c in "abcdefghijklmnopqrstuvwxyz") / 26.0
    max_chars = max(10, int((W * 0.9) / max(1, avg_char_w)))
    wrapped = textwrap.fill(text, width=max_chars)

    tw, th = draw.multiline_textsize(wrapped, font=font)
    x = (W - tw) / 2
    y = (H - th - padding) if position == 'bottom' else padding

    stroke = max(2, int(font_size * 0.06))
    for dx in range(-stroke, stroke+1):
        for dy in range(-stroke, stroke+1):
            if dx == 0 and dy == 0: continue
            draw.multiline_text((x+dx, y+dy), wrapped, font=font, fill=(0,0,0,200), align='center')

    draw.multiline_text((x, y), wrapped, font=font, fill=(255,255,255,255), align='center')
    return img.convert("RGB")


In [ ]:
def generate_captions_gemini(keywords, n=3, model="gemini-1.5-flash"):
    prompt = f"Generate {n} short, punchy captions for: {keywords}. Keep 1-2 lines."
    try:
        resp = client.models.generate_content(model=model, contents=prompt)
        text = resp.text or ""
        lines = [re.sub(r'^[\-\d\.\)\s]+', '', ln).strip() for ln in text.splitlines() if ln.strip()]
        if len(lines) < n:
            parts = [p.strip() for p in re.split(r'[。\n.!?]+', text) if p.strip()]
            for p in parts:
                if p not in lines:
                    lines.append(p)
        return lines[:n]
    except Exception as e:
        print("Error generating captions:", type(e).__name__, str(e))
        return [keywords]


In [ ]:
def create_meme_from_image(pil_img, keywords, style='meme'):
    captions = generate_captions_gemini(keywords, n=3)
    chosen = captions[0] if captions else keywords
    font = FONT_BANGERS if style == 'meme' else FONT_MONTSERRAT
    pos = 'bottom' if style == 'meme' else 'top'
    return draw_text_on_image(pil_img.copy(), chosen, font_path=font, position=pos), captions


In [ ]:
from IPython.display import display

resp = requests.get("https://picsum.photos/900/600", stream=True)
img = Image.open(resp.raw).convert("RGB")

result_img, caption_options = create_meme_from_image(img, "college fest music food friends", style='meme')
print("Caption options:", caption_options)
display(result_img)
